In [ ]:
from reprojection import reproject
import rioxarray
import geopandas as gpd
import xarray as xr
import datashader as ds
from datashader.transfer_functions import shade, stack
from datashader.colors import Elevation
from xrspatial import hillshade
import spatialpandas as spd
from rasterio.features import rasterize
from datashader.transfer_functions import dynspread

In [ ]:
%config Completer.use_jedi = False

### Reprojection: transforming map DataArrays between coordinate reference systems (crs)
#### Xarray-spatial's reproject takes in a spatially referenced source raster aligned with one coordinate reference system (crs) and transforms it into another crs, warping the shape and features.

For what follows, we'll first set up a shader function to render our output DataArrays in color.

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
countries = world[['name', 'geometry']]
# countries = countries.to_crs('EPSG:3857')
# countries = countries.explode('geometry').reset_index(drop=True)
countries = spd.GeoDataFrame(countries, geometry='geometry')
countries

In [ ]:
earth_lat_lon = rioxarray.open_rasterio('./elevation.tif').squeeze()
shade(earth_lat_lon)

In [ ]:
elev2D = earth_lat_lon
da_height, da_width = elev2D.shape
xmin, xmax = elev2D.coords['x'].data.min(), elev2D.coords['x'].data.max()
ymin, ymax = elev2D.coords['y'].data.min(), elev2D.coords['y'].data.max()
canvas = ds.Canvas(plot_height=da_height, plot_width=da_width, x_range=(xmin, xmax), y_range=(ymin, ymax))

countries_pgns = canvas.polygons(countries, geometry='geometry')

elev_raster = canvas.raster(elev2D)
shade(elev_raster)
countries_pgns.coords['x'] = elev_raster.coords['x']
countries_pgns.coords['y'] = elev_raster.coords['y'] 
countries_pgns.data = ds.utils.orient_array(countries_pgns)
# elev_raster.data = ds.utils.orient_array(elev_raster)
shade_elev = shade(elev_raster,alpha=255)
shade_countries = shade(countries_pgns, cmap=['red'], alpha=50)
stack(shade_elev, shade_countries)


In [ ]:

def shade_elev_hs(elev_3D, band=1):
    if not isinstance(elev_3D, xr.DataArray):
        elev_3D = xr.DataArray(elev_3D, dims=('band', 'y', 'x'))
    return stack(shade(elev_3D[band-1], cmap=Elevation),
                 shade(hillshade(elev_3D[band-1]), cmap=['black', 'white'],
                       how='eq_hist', alpha=0.6))

Now, we're ready to generate some map images.

Representing a 3D surface like the earth in a 2D map has been a challenge for a long time in cartography. Many solutions have been found, but ultimately it comes down to a compromise between distortions in distance, direction, and area. Different coordinate reference systems (crs) perform differently in these 3 metrics in different map areas.

A few of the more well-known projections follow:

The lat/lon (equirectangular) projection, EPSG:4326.
This very common one maps meridians and parallels to equally spaced vertical and horizontal lines, respectfully. Conversion between x, y coordinates on the map and earth locations are very simple, but its significant distortions make it impractical for many uses.

We'll open an earth elevation map in this crs.

In [ ]:
earth_lat_lon = rioxarray.open_rasterio('/Users/ls/Downloads/elevation.tif', chunks=25)
shade_elev_hs(earth_lat_lon)

Now, we'll use Xarray-spatial's `reproject` to reproject it into the web mercator crs. This is a cylindrical projection and produces a conformal map (true angles), and it is the projection of choice for almost all web maps.

To perform the reprojection, we just need the EPSG code, which is EPSG:4326 in this case. 

In [ ]:
web_mercator_crs = 'EPSG:3857'
web_mercator_earth = reproject(earth_lat_lon, web_mercator_crs)
shade_elev_hs(web_mercator_earth)

Next, we'll try the Robinson projection. This map was designed to be "right appearing" and has small distorions in all dimensions. It's code is ESRI:54030.

In [ ]:
robinson_crs = 'ESRI:54030'
robinson_earth = reproject(earth_lat_lon, robinson_crs)
shade_elev_hs(robinson_earth)

Another well-used one is the Transverse Mercator projection. It is a revision to the standard Mercator projection and also cylindrical and conformal, and it works best for mapping areas over small longitude range, such as states and counties.

In [ ]:
transverse_mercator_crs = 'EPSG:3004'
transverse_mercator_earth = reproject(earth_lat_lon, transverse_mercator_crs)
shade_elev_hs(transverse_mercator_earth)
# shade_elev_hs(earth_lat_lon.rio.reproject('EPSG:3004'))

In [ ]:
The Lambert Conformal Conic projection, which works best around middle meridians.

In [ ]:
lambert_crs = 'ESRI:102009'
lambert_earth_NA = reproject(earth_lat_lon, lambert_crs)
shade_elev_hs(lambert_earth_NA)

Another, unique, projection is the Space Oblique. This is a projection especially designed for satellite imagery so that it is completely free of distortion along the orbit path of the satellite.

In [ ]:
space_oblique_crs = 'EPSG:29873'
space_oblique_earth = reproject(earth_lat_lon, space_oblique_crs)
shade_elev_hs(space_oblique_earth)